In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [2]:
from transformers import pipeline

2023-03-20 15:11:23.565377: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.DataFrame()
df = pd.read_csv("./label.csv")
# Add Code here to extract more data from the text (sentiment etc and add it to the assign function as a lambda. e.g. sentiment = lambda x: sentiment_func(x["TEXT"]) )
# for path in (Path.cwd() / "../Dataset").glob("**/*.xml"):
#    year, manufacturer, *model = path.stem.split("_")
#    try:
#        with open(path, "r", encoding="unicode_escape") as f:
#            data = f.readlines()
#        data = "<root>" + "".join(data[1:]) + "</root>"
#        data = data.replace("&", "&amp;")
#        temp = pd.read_xml(data)
#        temp = temp.assign(year=year, manufacturer=manufacturer, model=" ".join(model))
#        df = pd.concat([df, temp])
        # Preprocess text (username and link placeholders)

#    except Exception as e:
#        print(path.stem, e)

In [4]:
df.rename(columns={"Unnamed: 0": "Initial Index"}, inplace= True)

In [5]:
print(df.columns)

Index(['Initial Index', 'DATE', 'AUTHOR', 'TEXT', 'FAVORITE', 'year',
       'manufacturer', 'model', 'label'],
      dtype='object')


In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = df['TEXT'].tolist()[0:10]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [14]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
# def preprocess(text):
#    new_text = []
#    for t in text.split(" "):
#        t = '@user' if t.startswith('@') and len(t) > 1 else t
#        t = 'http' if t.startswith('http') else t
#        new_text.append(t)
#    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
scores_posneg = []
for x in range(df.shape[0]):
    text = df.iloc[x:x+1]['TEXT'].values[0]
    # text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
#    for i in range(scores.shape[0]):
#        l = config.id2label[ranking[i]]
#        s = scores[ranking[i]]
        # pos, neutral, neg
    if (config.id2label[ranking[0]] == "positive"):
            scores_posneg.append(2)
    elif(config.id2label[ranking[0]] == "neutral"):
        scores_posneg.append(1)
    else:
        scores_posneg.append(0)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2    2010
0     281
1     109
dtype: int64


In [16]:
df_scores_posneg = pd.DataFrame(scores_posneg)
df['labels_posneg'] = scores_posneg

In [17]:
df.head()

,Initial Index,DATE,AUTHOR,TEXT,FAVORITE,year,manufacturer,model,label,labels_posneg
0,18455,08/27/2008,tjl,Owned this 07 GL320 CDI for 15 months. Overall...,"Great ride. Great handling. Goes a long, long ...",2007,mercedes-benz,gl-class,4,2
1,9534,04/07/2008,Brad,This is my fourth Explorer and they just keep ...,"Sync, Sirius, Class 3/4 trailer hitch set-up w...",2008,ford,explorer,5,2
2,18250,05/31/2007,Tib=Fun!,"If you like the way it looks, you will love th...",The blue backlit displays are awesome. The sou...,2007,hyundai,tiburon,5,2
3,7395,12/28/2008,crasberry,I realized in the 1st tank of gas that I wasn'...,The vehicle drives great and has plenty of roo...,2008,saturn,outlook,3,0
4,22773,01/17/2007,Eric H.,Traded in a 06 Honda pilot. Now I have an RX 3...,Power tailgate & the HID light.,2007,lexus,rx 350,4,2


In [18]:
df.shape

(2400, 10)

In [19]:
df.to_csv("label_output.csv", index=False)

In [ ]:
df["manufacturer"].value_counts()